In [1]:
#r "nuget:CardanoSharp.Wallet,2.14.0"
#r "nuget:CardanoSharp.Koios.Client,1.0.8"
#r "nuget:Refit,6.3.2"

Installed Packages CardanoSharp.Koios.Client, 1.0.8 CardanoSharp.Wallet, 2.14.0 Refit, 6.3.2

## Summary

We are going to create 2 separate wallets. Then we will take both wallets' Zero Index Payment Public Key and combine to create a MultiSig/NativeScript address. We will use the `ALL` Policy requiring both wallets to sign transactions for the MultiSig Wallet.

### Step 1: Create Wallets

In [2]:
using CardanoSharp.Wallet;
using CardanoSharp.Wallet.Enums;
using CardanoSharp.Wallet.Extensions.Models;
using CardanoSharp.Wallet.Models.Addresses;
using CardanoSharp.Wallet.Models.Keys;
using CardanoSharp.Wallet.Models.Derivations;

//Wallet 1
Mnemonic mnemonic1 = new MnemonicService().Restore("scale fiction sadness render fun system hunt skull awake neither quick uncle grab grid credit");
//Derive Payment Public Key
IIndexNodeDerivation paymentNode1 = mnemonic1.GetMasterNode()
    .Derive(PurposeType.MultiSig)
    .Derive(CoinType.Ada)
    .Derive(0)
    .Derive(RoleType.ExternalChain)
    .Derive(0);
paymentNode1.SetPublicKey();
//Derive Stake Public Key
IIndexNodeDerivation stakeNode1 = mnemonic1.GetMasterNode()
    .Derive(PurposeType.MultiSig)
    .Derive(CoinType.Ada)
    .Derive(0)
    .Derive(RoleType.Staking)
    .Derive(0);
stakeNode1.SetPublicKey();

//Wallet 2
Mnemonic mnemonic2 = new MnemonicService().Restore("harsh absorb lazy resist elephant social carry roof remember picture merry enlist regret major practice");
//Derive Payment Public Key
IIndexNodeDerivation paymentNode2 = mnemonic2.GetMasterNode()
    .Derive(PurposeType.MultiSig)
    .Derive(CoinType.Ada)
    .Derive(0)
    .Derive(RoleType.ExternalChain)
    .Derive(0);
paymentNode2.SetPublicKey();
//Derive Stake Public Key
var stakeNode2 = mnemonic2.GetMasterNode()
    .Derive(PurposeType.MultiSig)
    .Derive(CoinType.Ada)
    .Derive(0)
    .Derive(RoleType.Staking)
    .Derive(0);
stakeNode2.SetPublicKey();

//Important Values
PublicKey paymentPub1 = paymentNode1.PublicKey;
PublicKey stakePub1 = stakeNode1.PublicKey;
PublicKey paymentPub2 = paymentNode2.PublicKey;
PublicKey stakePub2 = stakeNode2.PublicKey;

### Step 2: Create Policy Script for MultiSig

If you've minting assets before, this will look familiar.

In [3]:
using CardanoSharp.Wallet.Utilities;
using CardanoSharp.Wallet.TransactionBuilding;
using CardanoSharp.Wallet.Models.Transactions.Scripts;
using CardanoSharp.Wallet.Extensions;

// Generate payment hashes
byte[] paymentHash1 = HashUtility.Blake2b224(paymentPub1.Key);
byte[] paymentHash2 = HashUtility.Blake2b224(paymentPub2.Key);

// Create a Payment Policy Script with a type of Script Any
IScriptAllBuilder paymentPolicyScriptBuilder = ScriptAllBuilder.Create
    .SetScript(NativeScriptBuilder.Create.SetKeyHash(paymentHash1))
    .SetScript(NativeScriptBuilder.Create.SetKeyHash(paymentHash2));

ScriptAll paymentPolicyScript = paymentPolicyScriptBuilder.Build();
byte[] paymentPolicyId = paymentPolicyScript.GetPolicyId();

// Generate stake hashes
byte[] stakeHash1 = HashUtility.Blake2b224(stakePub1.Key);
byte[] stakeHash2 = HashUtility.Blake2b224(stakePub2.Key);

// Create a Stake Policy Script with a type of Script Any
IScriptAllBuilder stakePolicyScriptBuilder = ScriptAllBuilder.Create
    .SetScript(NativeScriptBuilder.Create.SetKeyHash(stakeHash1))
    .SetScript(NativeScriptBuilder.Create.SetKeyHash(stakeHash2));

ScriptAll stakePolicyScript = stakePolicyScriptBuilder.Build();
byte[] statkePolicyId = stakePolicyScript.GetPolicyId();

// template if we want to verify our Native Script 
// {
//     "type": "all",
//     "scripts": [
//         {
//             "type": "sig",
//             "keyHash": ""
//         },
//         {
//             "type": "sig",
//             "keyHash": ""
//         }
//     ]
// }

### Step 3: Create MultiSig Address

Take the address and send some test ADA to it. Network is up to you.

In [4]:
using CardanoSharp.Wallet.Models;
using CardanoSharp.Wallet;

var multiSigAddress = new AddressService().GetBaseScriptAddress(paymentPolicyScript, stakePolicyScript, NetworkType.Testnet);
Console.WriteLine(multiSigAddress);

addr_test1xqg0c32s2a9wyxt9ljz88raatvks9r0yejxztvd535lykg8f6xl537h7rscchceanuzl5vt3xlkp6pp69pl7qsaez49qgxvj5g


### Step 4: Check Balance

I added an address that isn't derived from a policy so we can see the distinction.

In [8]:
using CardanoSharp.Koios.Client;
using CardanoSharp.Wallet.CIPs.CIP2;
using CardanoSharp.Wallet.CIPs.CIP2.Models;
using CardanoSharp.Wallet.Models.Transactions;
using CardanoSharp.Wallet.Extensions;
using CardanoSharp.Wallet.Extensions.Models.Transactions;
using CardanoSharpAsset = CardanoSharp.Wallet.Models.Asset;
using Refit;

var addressClient = RestService.For<IAddressClient>("https://testnet.koios.rest/api/v0");

var addressBulkRequest = new AddressBulkRequest { Addresses = new List<string> { multiSigAddress.ToString(), "addr_test1qzx9hu8j4ah3auytk0mwcupd69hpc52t0cw39a65ndrah86djs784u92a3m5w475w3w35tyd6v3qumkze80j8a6h5tuqq5xe8y" } };
var addressInformation = addressClient.GetAddressInformation(addressBulkRequest).Result;
foreach (var ai in addressInformation.Content)
{
    Console.WriteLine($"Address: {ai.Address}");
    Console.WriteLine($"Balance: {ai.Balance}, Is Script?: {ai.ScriptAddress}");
    Console.WriteLine("");
}

//Helper Functions
public async Task<List<Utxo>> GetUtxos(string address)
{
    try
    {
        var addressBulkRequest = new AddressBulkRequest { Addresses = new List<string> { address } };
        var addressResponse = (await addressClient.GetAddressInformation(addressBulkRequest));
        var addressInfo = addressResponse.Content;
        var utxos = new List<Utxo>();

        foreach (var ai in addressInfo.SelectMany(x => x.UtxoSets))
        {
            if(ai is null) continue;
            var utxo = new Utxo()
            {
                TxIndex = ai.TxIndex,
                TxHash = ai.TxHash,
                Balance = new Balance()
                {
                    Lovelaces = ulong.Parse(ai.Value)
                }
            };

            var assetList = new List<CardanoSharpAsset>();
            foreach (var aa in ai.AssetList)
            {
                assetList.Add(new CardanoSharpAsset()
                {
                    Name = aa.AssetName,
                    PolicyId = aa.PolicyId,
                    Quantity = ulong.Parse(aa.Quantity)
                });
            }

            utxo.Balance.Assets = assetList;
            utxos.Add(utxo);
        }

        return utxos;
    }
    catch (Exception ex)
    {
        Console.WriteLine(ex.Message);
        return null;
    }
}

private void AddInputsFromCoinSelection(CoinSelection coinSelection, ITransactionBodyBuilder transactionBody)
{
    foreach (var i in coinSelection.Inputs)
    {
        transactionBody.AddInput(i.TransactionId, i.TransactionIndex);
    }
}

private void AddChangeOutputs(ITransactionBodyBuilder ttb, List<TransactionOutput> outputs, string address)
{
    foreach (var output in outputs)
    {
        ITokenBundleBuilder? assetList = null;

        if (output.Value.MultiAsset is not null)
        {
            assetList = TokenBundleBuilder.Create;
            foreach (var ma in output.Value.MultiAsset)
            {
                foreach (var na in ma.Value.Token)
                {
                    assetList.AddToken(ma.Key, na.Key, na.Value);
                }
            }
        }

        ttb.AddOutput(new Address(address), output.Value.Coin, assetList);
    }
}

Address: addr_test1qzx9hu8j4ah3auytk0mwcupd69hpc52t0cw39a65ndrah86djs784u92a3m5w475w3w35tyd6v3qumkze80j8a6h5tuqq5xe8y
Balance: 396602216, Is Script?: False

Address: addr_test1xqg0c32s2a9wyxt9ljz88raatvks9r0yejxztvd535lykg8f6xl537h7rscchceanuzl5vt3xlkp6pp69pl7qsaez49qgxvj5g
Balance: 19824379, Is Script?: True



### Step 5: Build Transaction to Send from MultiSig Address

In [18]:
using System.IO;
using Newtonsoft.Json;
using CardanoSharp.Wallet.CIPs.CIP2;
using CardanoSharp.Wallet.CIPs.CIP2.Models;
using CardanoSharp.Wallet.Models.Transactions;
using CardanoSharp.Wallet.Extensions;
using CardanoSharp.Wallet.Extensions.Models.Transactions;
using CardanoSharpAsset = CardanoSharp.Wallet.Models.Asset;
using CardanoSharp.Koios.Client;
using Refit;

//setup koios clients
var addressClient = RestService.For<IAddressClient>("https://testnet.koios.rest/api/v0");
var networkClient = RestService.For<INetworkClient>("https://testnet.koios.rest/api/v0");
var epochClient = RestService.For<IEpochClient>("https://testnet.koios.rest/api/v0");

//1. Get UTxOs
var utxos = await GetUtxos(multiSigAddress.ToString());

///2. Create the Body
var transactionBody = TransactionBodyBuilder.Create;

//set outputs
transactionBody.AddOutput("addr_test1qzkllrylzkgu6gh6ltauvepxsp8hl3awjhz0208q2eu2hk9g8tv23d70fh94ehd40t0ew8ztt2zrgcct2ttal7jmjehsfe79m2".ToAddress().GetBytes(), 5000000);

//perform coin selection
var coinSelection = ((TransactionBodyBuilder)transactionBody).UseLargestFirstWithImprove(utxos);

//add the inputs from coin selection to transaction body builder
AddInputsFromCoinSelection(coinSelection, transactionBody);

//if we have change from coin selection, add to outputs
if (coinSelection.ChangeOutputs is not null && coinSelection.ChangeOutputs.Any())
{
    AddChangeOutputs(transactionBody, coinSelection.ChangeOutputs, multiSigAddress.ToString());
}

//get protocol parameters and set default fee
var epochResponse = await epochClient.GetEpochInformation();
var ppResponse = await epochClient.GetProtocolParameters();
var protocolParameters = ppResponse.Content.FirstOrDefault();
transactionBody.SetFee(protocolParameters.MinFeeB.Value);

//get network tip and set ttl
var blockSummaries = (await networkClient.GetChainTip()).Content;
var ttl = 2500 + (uint)blockSummaries.First().AbsSlot;
transactionBody.SetTtl(ttl);

///3. Add Witnesses
var witnessSet = TransactionWitnessSetBuilder.Create;
witnessSet.SetScriptAllNativeScript(paymentPolicyScriptBuilder);
witnessSet.AddVKeyWitness(paymentNode1.PublicKey, paymentNode1.PrivateKey);
witnessSet.AddVKeyWitness(paymentNode2.PublicKey, paymentNode2.PrivateKey);

///4. Build Draft TX
//create transaction builder and add the pieces
var transaction = TransactionBuilder.Create;
transaction.SetBody(transactionBody);
transaction.SetWitnesses(witnessSet);

//get a draft transaction to calculate fee
var draft = transaction.Build(); 
var fee = draft.CalculateFee(protocolParameters.MinFeeA, protocolParameters.MinFeeB);

//update fee and change output
transactionBody.SetFee(fee);
var raw = transaction.Build();
raw.TransactionBody.TransactionOutputs.Last().Value.Coin -= fee;
Console.WriteLine(raw.Serialize().ToStringHex());


84a4008182582046fe9439b620443c5fa3cae06c005ab1d463aa4f7782428eef8daf7842196cba01018282583900adff8c9f1591cd22fafafbc66426804f7fc7ae95c4f53ce05678abd8a83ad8a8b7cf4dcb5cddb57adf971c4b5a8434630b52d7dffa5b966f1a004c4b408258393010fc4550574ae21965fc84738fbd5b2d028de4cc8c25b1b48d3e4b20e9d1bf48fafe1c318be33d9f05fa317137ec1d043a287fe043b9154a1a00df85b6021a0002ae05031a044723c3a20082825820966d876075002a9f80b64037c12c9c5260f39024d86c29c667fa148923a0a52a584041c9f200223bd11587d51bd4f2da45e1b9bd1ce956131faf95fd20f4b0e55a947e6edd8202ef0057d02fa313c79934b8b5259e7efd6bbc2d9b1b816decd8e70e82582080bb781421b521a5c0e99042f2b05d8ea5c876d8e3f700de00b6fe8cd7cff87b5840af41dd58bd320b0719446fe6ae87a2b3a1415e0b040662419490082e5148bb3a2d787371610b7e7d8e880587a63420f6835408786d2e2c152bf473576852530501818201828200581ccf66e4dfee27510a5263d60bbb7e371be1133ea365b8c578b3098c2e8200581c0b93a74a952be650d58ced2283e045465dc4f3881077ebb647b2c79cf5f6


### Step 6: Submit Transaction

In [45]:
using CardanoSharp.Wallet.Extensions;
var transactionClient = RestService.For<ITransactionClient>("https://testnet.koios.rest/api/v0");

var signed = raw.Serialize();
try {
    using (MemoryStream stream = new MemoryStream(signed))
    {
        try
        {
            Console.WriteLine("Sending...");
            var result = transactionClient.Submit(stream).Result;
            Console.WriteLine($"Tx ID: {result.Content}");
        }
        catch (Exception e)
        {
            Console.Write(e.Message);
        }
    }
}
catch(Exception e) 
{
    Console.WriteLine(e.Message);
}

{"TransactionBody":{"TransactionInputs":[{"TransactionId":"qAbs1YdDp5alPQaUcBly6D4/N7IYkpLN+O5hXvvZ+Sg=","TransactionIndex":0}],"TransactionOutputs":[{"Address":"AK3/jJ8Vkc0i+vr7xmQmgE9/x66VxPU84FZ4q9ioOtiot89Ny1zdtXrflxxLWoQ0YwtS19/6W5Zv","Value":{"Coin":5000000,"MultiAsset":null}},{"Address":"MBD8RVBXSuIZZfyEc4+9Wy0CjeTMjCWxtI0+SyDp0b9I+v4cMYvjPZ8F+jFxN+wdBDoof+BDuRVK","Value":{"Coin":19824379,"MultiAsset":null}}],"Fee":175621,"Ttl":71649626,"Certificate":null,"Withdrawls":null,"Update":null,"MetadataHash":null,"TransactionStartInterval":null,"Mint":{}},"TransactionWitnessSet":{"VKeyWitnesses":[{"VKey":{"Key":"lm2HYHUAKp+AtkA3wSycUmDzkCTYbCnGZ/oUiSOgpSo=","Chaincode":null},"SKey":null,"Signature":"lsyRT6wsdDe52KPQuyanryUTW0U2gqyUZzOYh7aDPj5BkfLuGsUvuv57qeRS9n5qpBYcbRDca5r6D3N36sMUBg==","IsMock":false},{"VKey":{"Key":"gLt4FCG1IaXA6ZBC8rBdjqXIdtjj9wDeALb+jNfP+Hs=","Chaincode":null},"SKey":null,"Signature":"/sMDan3Nx9ggdjvnjeGUqbqicqO9+TtjMXvpSByqsm+z8u7ev2s7Eet3oiyY9yNtWJpZTt14P39QX+dy

### Bonus: Realistic Example

Lets assume that the transaction needs to be passed around to be signed. We'll even assume that our initial transaction was built by neither party who needs to sign.

In [27]:
using System.IO;
using Newtonsoft.Json;
using CardanoSharp.Wallet.CIPs.CIP2;
using CardanoSharp.Wallet.CIPs.CIP2.Models;
using CardanoSharp.Wallet.Models.Transactions;
using CardanoSharp.Wallet.Extensions;
using CardanoSharp.Wallet.Extensions.Models.Transactions;
using CardanoSharpAsset = CardanoSharp.Wallet.Models.Asset;
using CardanoSharp.Koios.Client;
using Refit;

//setup koios clients
var addressClient = RestService.For<IAddressClient>("https://testnet.koios.rest/api/v0");
var networkClient = RestService.For<INetworkClient>("https://testnet.koios.rest/api/v0");
var epochClient = RestService.For<IEpochClient>("https://testnet.koios.rest/api/v0");

//1. Get UTxOs
var utxos = await GetUtxos(multiSigAddress.ToString());

///2. Create the Body
var transactionBody = TransactionBodyBuilder.Create;

//set outputs
transactionBody.AddOutput("addr_test1qzkllrylzkgu6gh6ltauvepxsp8hl3awjhz0208q2eu2hk9g8tv23d70fh94ehd40t0ew8ztt2zrgcct2ttal7jmjehsfe79m2".ToAddress().GetBytes(), 5000000);

//perform coin selection
var coinSelection = ((TransactionBodyBuilder)transactionBody).UseLargestFirstWithImprove(utxos);

//add the inputs from coin selection to transaction body builder
AddInputsFromCoinSelection(coinSelection, transactionBody);

//if we have change from coin selection, add to outputs
if (coinSelection.ChangeOutputs is not null && coinSelection.ChangeOutputs.Any())
{
    AddChangeOutputs(transactionBody, coinSelection.ChangeOutputs, multiSigAddress.ToString());
}

//get protocol parameters and set default fee
var epochResponse = await epochClient.GetEpochInformation();
var ppResponse = await epochClient.GetProtocolParameters();
var protocolParameters = ppResponse.Content.FirstOrDefault();
transactionBody.SetFee(protocolParameters.MinFeeB.Value);

//get network tip and set ttl
var blockSummaries = (await networkClient.GetChainTip()).Content;
var ttl = 2500 + (uint)blockSummaries.First().AbsSlot;
transactionBody.SetTtl(ttl);

////////////////////////////////////////////////////////////////
///3. Mock Witnesses
//Change 1
var witnessSet = TransactionWitnessSetBuilder.Create
    .MockVKeyWitness(2);
////////////////////////////////////////////////////////////////

///4. Build Draft TX
//create transaction builder and add the pieces
var transaction = TransactionBuilder.Create;
transaction.SetBody(transactionBody);
transaction.SetWitnesses(witnessSet);

//get a draft transaction to calculate fee
var draft = transaction.Build(); 

////////////////////////////////////////////////////////////////
//Change 2
draft.TransactionWitnessSet.NativeScripts.Add(
    new NativeScript() {
        ScriptAll = paymentPolicyScriptBuilder.Build()
    });
////////////////////////////////////////////////////////////////

var fee = draft.CalculateFee(protocolParameters.MinFeeA, protocolParameters.MinFeeB);

//update fee and change output
transactionBody.SetFee(fee);
var rawMocked = transaction.Build();
rawMocked.TransactionBody.TransactionOutputs.Last().Value.Coin -= fee;


Now who ever generated the unsigned transaction will send it to actor 1 and actor 2.

In [28]:
//Hash the tx body to sign
var txBodyHash = HashUtility.Blake2b256(rawMocked.TransactionBody.GetCBOR(rawMocked.AuxiliaryData).EncodeToBytes());

//First Key Sign
var vkeyWitness1 = new VKeyWitness(){
    VKey = paymentNode1.PublicKey,
    Signature = paymentNode1.PrivateKey.Sign(txBodyHash)
};

//Second Key Sign
var vkeyWitness2 = new VKeyWitness(){
    VKey = paymentNode2.PublicKey,
    Signature = paymentNode2.PrivateKey.Sign(txBodyHash)
};

Once both actors sign the transaction, they send back to the transaction builder for submission.

In [29]:
//get network tip and set ttl
var blockSummaries = (await networkClient.GetChainTip()).Content;
var ttl = 2500 + (uint)blockSummaries.First().AbsSlot;
rawMocked.TransactionBody.Ttl = ttl;

//reset and add signatures
rawMocked.TransactionWitnessSet.VKeyWitnesses.Clear();
rawMocked.TransactionWitnessSet.VKeyWitnesses.Add(vkeyWitness1);
rawMocked.TransactionWitnessSet.VKeyWitnesses.Add(vkeyWitness2);

//submit tx
var transactionClient = RestService.For<ITransactionClient>("https://testnet.koios.rest/api/v0");

var signed = rawMocked.Serialize();
Console.WriteLine(signed.ToStringHex());
try {
    using (MemoryStream stream = new MemoryStream(signed))
    {
        try
        {
            Console.WriteLine("Sending...");
            var result = transactionClient.Submit(stream).Result;
            if(result.Content is not null)
                Console.WriteLine($"Tx ID: {result.Content}");
            else 
                Console.WriteLine(result.Error.Content);
        }
        catch (Exception e)
        {
            Console.Write(e.Message);
        }
    }
}
catch(Exception e) 
{
    Console.WriteLine(e.Message);
}

84a4008182582046fe9439b620443c5fa3cae06c005ab1d463aa4f7782428eef8daf7842196cba01018282583900adff8c9f1591cd22fafafbc66426804f7fc7ae95c4f53ce05678abd8a83ad8a8b7cf4dcb5cddb57adf971c4b5a8434630b52d7dffa5b966f1a004c4b408258393010fc4550574ae21965fc84738fbd5b2d028de4cc8c25b1b48d3e4b20e9d1bf48fafe1c318be33d9f05fa317137ec1d043a287fe043b9154a1a00df85b6021a0002ae05031a04472644a20082825820966d876075002a9f80b64037c12c9c5260f39024d86c29c667fa148923a0a52a5840f8ddb91d568b52d3a7af30975d27553a73423e40ba2f41d9c0de75c8ecda0588b062f6455dd8292f35ffbae7d94da05f4a68e5d4933b76b20cbc25134d6e7b0382582080bb781421b521a5c0e99042f2b05d8ea5c876d8e3f700de00b6fe8cd7cff87b5840e044fada57427526d59f6c3a4e71ba00e90232cecbfd23506d25fbb6ab056bde6928d6c128cb3b3ff647a02b2d89d6128a8581fa0c87d7cab364612a160fa00901818201828200581ccf66e4dfee27510a5263d60bbb7e371be1133ea365b8c578b3098c2e8200581c0b93a74a952be650d58ced2283e045465dc4f3881077ebb647b2c79cf5f6
Sending...
Tx ID: "c98b2176635c3834cbd05cfeec8cb88d899cec63cebe00f97cbcd26c0eb8